In [1]:
import numpy as np
from scipy import stats
import multiprocessing
import os
import pandas as pd
from parallel_function import KDE
import time
import pickle
import json
import pandas as pd

In [2]:
def get_files_and_info(worker,ipath):
    n_expected=7 # header + folder [0-5]
    results={}
    npath=ipath+worker
    sessions=os.listdir(npath)
    sessionID=-1
    m=""
    if len(sessions) == 1:
        npath=ipath+"{}/{}".format(worker,sessions[0])
        if len(os.listdir(npath)) ==7:
            sessionID=sessions[0]
            m+="OK "
        else:
            m+="SingleSession Incomplete "
    else:
        success=0
        m+="Multiple Sessions "
        for s in sessions:
            npath=ipath+"{}/{}".format(worker,s)
            if len(os.listdir(npath)) ==7:
                success+=1
                sessionID=s
        if success == 1:
            m+="one OK "
        else:
            sessionID=-1
            m+=str(success)+" Times completed "
            print(npath, m)
    if sessionID!=-1:
        npath=ipath+"{}/{}/".format(worker,sessionID)
        fname="{}_header.json".format(worker,sessionID,worker)
        with open(npath+fname, 'r') as f:
            parsed_json = json.load(f)
        fileList=[]
        
        for tnum in os.listdir(npath):
            trial={}
            test_path = os.path.join(npath, tnum)
            if os.path.isdir(test_path):
                filename="{}_trial_{}_CORRECT.csv".format(worker,tnum)
                if os.path.isfile(os.path.join(test_path, filename)):
                    header="{}_trial_{}_PH.json".format(worker,tnum)
                    tail="{}_trial_{}_PT.json".format(worker,tnum)
                    with open(os.path.join(test_path, header), 'r') as f:
                        parsed_json1 = json.load(f)
                    with open(os.path.join(test_path, tail), 'r') as f:
                        parsed_json2 = json.load(f)
                    meta = {**parsed_json1 , **parsed_json2}                
                    fileList.append({"trial":int(tnum),"path": test_path,"filename":filename,"meta":meta})
        results['info']=parsed_json
        results['files']=fileList
    results['state']=m
    return results

In [3]:
def ProcessDataframe(fname):
    df=pd.read_csv(fname,names=["Time", "x", "y", "z","Azimuth","Elevation"])
    df = df.apply(pd.to_numeric, errors='coerce')
    df = df.dropna()
    df = df.drop('Time', 1)
    df = df.drop('Azimuth', 1)
    df = df.drop('Elevation', 1)
    return df

In [6]:
allpaths=["../../results/"]

In [7]:
all_merged=[pd.DataFrame() for i in range(3)]
for ipath in allpaths:
    print(ipath)
    ignored=0
    
    workerIDs=os.listdir(ipath)
    entry_num=0
    for worker in workerIDs:
        entry=get_files_and_info(worker,ipath)
        if 'info' in entry:
            groupIDX=int(entry['info']['gr'])-1
            for file in entry['files']:
                filename=file['filename']
                fname=os.path.join(file['path'], filename)
                df=ProcessDataframe(fname)
                df['task']=int(file['meta']['td'])
                df['entry_num']=entry_num
                entry_num+=1
                all_merged[groupIDX]=all_merged[groupIDX].append(df,ignore_index=True)
        else:
            ignored+=1
all_merged[0].to_csv("VR_merged_data_BASE.csv",index=False)
all_merged[1].to_csv("VR_merged_data_ATRIA.csv",index=False)
all_merged[2].to_csv("VR_merged_data_GLASS.csv",index=False)
print("Total Ignored",ignored)

../../results/


NotADirectoryError: [Errno 20] Not a directory: '../../results/Untitled.ipynb'